In [ ]:
import sys
import asyncio
import time

import numpy as np

from lsst.ts import salobj

from lsst.ts.standardscripts.auxtel.attcs import ATTCS
from lsst.ts.standardscripts.auxtel.latiss import LATISS

In [ ]:
# Temporarily needed to run at summit
import os
os.environ["LSST_DDS_DOMAIN"] = 'lsatmcs'
os.environ["OSPL_URI"] = "file:///home/patrickingraham/ospl.xml"

In [ ]:
import logging
stream_handler = logging.StreamHandler(sys.stdout)
logger = logging.getLogger()
logger.addHandler(stream_handler)
logger.level = logging.DEBUG

In [ ]:
#get classes and start them
domain = salobj.Domain()
attcs = ATTCS(domain)
latiss = LATISS(domain)
await asyncio.gather(attcs.start_task, latiss.start_task)

In [ ]:
# enable components
#await attcs.enable()
#await latiss.enable()

In [ ]:
# take event checking out the slew commands to test telescope only
attcs.check.atdome = False
attcs.check.atdometrajectory = False

In [ ]:
# Perform Rotator verification (LTS-337-003)

In [ ]:
await attcs.stop_tracking()

In [ ]:
start_az=165.5
start_el=67.9
start_rot_pa=-170
await attcs.point_azel(start_az, start_el, rot_pa=start_rot_pa, wait_dome=True)

In [ ]:
start_az=165.5
start_el=67.9
start_rot_pa=170

start_time =time.time()
await attcs.point_azel(start_az, start_el, rot_pa=start_rot_pa, wait_dome=True)
end_time = time.time()
print(f'Time to perform {170*2} degree rotation of N2 is {end_time-start_time} seconds')

In [ ]:
# send telescope to alt/az/
#await salobj.set_summary_state(latiss.atcamera, salobj.State.ENABLED)

In [ ]:
# point telescope to desired starting position
# this make take extra time as more checks are performed
start_az=50
start_el=20
start_rot_pa=0
await attcs.point_azel(start_az, start_el, rot_pa=start_rot_pa, wait_dome=True)

In [ ]:
#declare offset size
d_az= 3.5 # degrees
d_el = 3.5 # degrees
d_rot= 3.5 # degrees

d_az= 10 # degrees
d_el = 10 # degrees
d_rot= 10 # degrees

d_az= 90 # degrees
d_el = 60 # degrees
d_rot= 90 # degrees

In [ ]:
from astropy.time import Time
from astropy.coordinates import AltAz, ICRS, EarthLocation, Angle, FK5
import astropy.units as u
location = EarthLocation.from_geodetic(lon=-70.747698*u.deg,
                                       lat=-30.244728*u.deg,
                                       height=2663.0*u.m)

In [ ]:
# get RA/DEC of current position
az = Angle(start_az, unit=u.deg)
el = Angle(start_el, unit=u.deg)
print(f'orig az {az} and el {el}')
time_data = await attcs.atptg.tel_timeAndDate.next(flush=True, timeout=2)
curr_time_atptg = Time(time_data.tai, format="mjd", scale="tai")
#print(curr_time_atptg)
coord_frame_AltAz = AltAz(location=location, obstime=curr_time_atptg)
coord_frame_radec = ICRS()
coord_azel = AltAz(az=az, alt=el, location=location, obstime=curr_time_atptg)
ra_dec = coord_azel.transform_to(coord_frame_radec)
print('Current Position is: \n {}'.format(coord_azel))
print('Current Position is: \n {}'.format(ra_dec))

# get RA/DEC of target position
az = Angle(start_az+d_az, unit=u.deg)
el = Angle(start_el+d_el, unit=u.deg)
print(f'target az {az} and el {el}')
coord_azel_target = AltAz(az=az, alt=el, location=location, obstime=curr_time_atptg)
ra_dec_target = coord_azel_target.transform_to(coord_frame_radec)
print('Target Position is: \n {}'.format(coord_azel_target))
print('Target Position is: \n {}'.format(ra_dec_target))


In [ ]:
# Why does doing no slew at all take ~32.7 sec?

#send to starting position
await attcs.slew_icrs(ra=str(ra_dec.ra), dec=str(ra_dec.dec), rot_pa=0.0, slew_timeout=240., stop_before_slew=True)


print('track for 2s')
await asyncio.sleep(2)
# take a quick image
await latiss.take_engtest(exptime=1)

print('Starting to Slew to target')
start_time = time.time()
await attcs.slew_icrs(ra=str(ra_dec_target.ra), dec=str(ra_dec_target.dec), rot_pa=d_rot, slew_timeout=240., stop_before_slew=False, wait_settle=False)
end_time = time.time()
print('Time to slew is {} [s]'.format(end_time-start_time))
await latiss.take_engtest(exptime=1)

In [ ]:
await attcs.stop_tracking()

In [ ]:
# Perform tracking test by taking continous 20s exposures
#await attcs.slew_object('HD 167060')
await attcs.slew_object('WD 1327-083')

In [ ]:
await latiss.take_object(exptime=15, grating='empty_1', filter='RG610')

In [ ]:
await latiss.take_object(exptime=15, grating='ronchi90lpmm', filter='empty_1')

In [ ]:
while True:
    await latiss.take_object(exptime=15, grating='empty_1', filter='RG610')
    await latiss.take_object(exptime=15, grating='ronchi90lpmm', filter='empty_1')

In [ ]:
# stopped tracking elsewhere, then reslewed
await attcs.slew_object('HD 59468')

In [ ]:
# OFFSET  AZ/EL
await attcs.offset_azel(az=-10*60*60, el=0)

In [ ]:
# again
await attcs.offset_azel(az=-20*60*60, el=0)

In [ ]:
# OFFSET  AZ/EL
await attcs.offset_azel(az=-10*60*60, el=0)

In [ ]:
# OFFSET  AZ/EL
await attcs.offset_azel(az=-5*60*60, el=0)
await attcs.offset_azel(az=-2*60*60, el=0)
await attcs.offset_azel(az=0, el=0)

In [ ]:
while True:
    await latiss.take_object(exptime=15, grating='empty_1', filter='RG610')
    await latiss.take_object(exptime=15, grating='ronchi90lpmm', filter='empty_1')
    

In [ ]:
await latiss.take_bias(nbias=1)

In [ ]:
#await attcs.startup()

In [ ]:
await latiss.enable()